In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

#Train IELM and evaluating training accuracy
no_classes = 8
error = 0.1
loss_function = "binary_cross_entropy" #"mean_squared_error"  #It can be mean_absolute_error also
activation_function = "tanh"
import tensorflow as tf

# Set the GPU memory growth option
physical_devices = tf.config.list_physical_devices("GPU")
print(physical_devices)
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)



In [ ]:
def loading_data(train_path,test_path):

    # Define the data transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to (224, 224)
        transforms.ToTensor(),  # Convert images to tensors
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
    ])

    

    train_dataset = ImageFolder(train_path, transform=transform)
    traindata_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

    test_dataset = ImageFolder(test_path, transform=transform)
    testdata_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)
    return traindata_loader,testdata_loader

def load_model():
    #loading of model
    device=torch.device("cuda")  
    
    vgg_model = models.vgg19(weights=models.VGG19_Weights.DEFAULT)
    vgg_model = torch.nn.Sequential(*list(vgg_model.features.children()))
    vgg_model=vgg_model.to(device)
    vgg_model.eval()
    return vgg_model
#training features extraction
def training_feature_extraction(device, traindata_loader,vgg_model):

    x_training_features = []
    y_train_labels = []
    with torch.no_grad():
        for images, labels in traindata_loader:        
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG16
            with torch.no_grad():
                features = vgg_model(images)   
            
            x_training_features.append(features)
            y_train_labels.append(labels)
    x_training_features = torch.cat(x_training_features, dim=0)
    y_train_labels = torch.cat(y_train_labels, dim=0)

    x_training_features.to(device)
    y_train_labels.to(device)
    return x_training_features,y_train_labels

def testing_feature_extratcion(device,testdata_loader,vgg_model):
    x_testing_features = []
    y_testing_labels = []
    with torch.no_grad():
        for images, labels in testdata_loader:
            images = images.to(device)
            labels = labels.to(device)
            # Extract features using VGG19
            with torch.no_grad():
                features = vgg_model(images) 
            x_testing_features.append(vgg_model(images))
            y_testing_labels.append(labels)
    x_testing_features = torch.cat(x_testing_features, dim=0)
    y_testing_labels = torch.cat(y_testing_labels, dim=0)

    x_testing_features.to(device)
    y_testing_labels.to(device)
    return x_testing_features,y_testing_labels


In [ ]:

"""
IELM Constructor class
"""
import torch
import torch.nn as nn
from sklearn.metrics import precision_recall_fscore_support
class I_ELM():
    """ Constructor to initialize node"""
    def __init__(self, no_input_nodes, max_no_hidden_nodes, no_output_nodes,
        activation_function='sigmoid', loss_function='mean_squared_error'):
        device=torch.device("cuda") 
        
        self.no_input_nodes = no_input_nodes
        self.no_hidden_nodes = 1
        self.no_output_nodes = no_output_nodes

        self.beta = torch.FloatTensor(self.no_hidden_nodes, self.no_output_nodes).uniform_(-1., 1.)
        self.beta=self.beta.to(device)
        
        # initialize weights between Input Layer and hidden layer
        self.alpha=torch.FloatTensor(self.no_input_nodes, self.no_hidden_nodes).uniform_(-1.,1.)
        self.alpha=self.alpha.to(device)
        
        # Initialize Biases
        self.bias =torch.zeros(size=(self.no_hidden_nodes,))
        self.bias=self.bias.to(device)
        # set an activation function
        self.activation_function = activation_function
       
        # set a loss function
        self.loss_function = loss_function
    
    def mean_squared_error(self,Y_True, Y_Pred):
        return 0.5 * torch.mean((Y_True - Y_Pred)**2)

    def mean_absolute_error(self, Y_True, Y_Pred):
        return torch.mean(torch.abs(Y_True - Y_Pred))
    
    def sigmoid(self, x):
        return 1. /(1.+ torch.exp(-x))
    
    def predict(self, X):
        return torch.tensor(self(X), dtype=torch.float32)
    
    def __call__(self, X):
        h = torch.sigmoid(torch.matmul(X,self.alpha) + self.bias)
        return torch.matmul(h,self.beta)
    
    def binary_cross_entropy(self,Y_true, Y_pred):
        lss=[]
        Y_pred = torch.clamp(Y_pred, min=1e-7, max=1.0 - 1e-7)
        loss_calculated=- (Y_true * torch.log(Y_pred) + (1 - Y_true) * torch.log(1 - Y_pred))
        '''loss_calculated_max=torch.argmax(loss_calculated, axis=-1)
        loss_calculated_max=loss_calculated_max.cpu()
        
        for i in range(len(loss_calculated_max)):
            l1=loss_calculated_max[i]
            lss.append(l1)
        #print(lss)
        plt.plot(range(1, Lmax), lss)
        plt.xlabel('Hidden Layer Neurons')
        plt.ylabel('Training Loss')
        #plt.title('BCE Loss Over Epochs')
        plt.show()
        '''
        loss=torch.mean(loss_calculated)
        return loss


    def plot_confusion(self,Y_true, Y_pred):
         
        # Calculate the confusion matrix 
        confusion = confusion_matrix(Y_true, Y_pred)
        
        # Extract values from the confusion matrix
        TP = confusion[1, 1]
        FP = confusion[0, 1]
        TN = confusion[0, 0]
        FN = confusion[1, 0]
        TPR= TP/(TP+FN)
        FPR=FP/(FP+TN)
        X=[0,FPR,1]
        Y=[0,TPR,1]
        AUCSCORE =np.trapz(Y,X)

        # Calculate precision, recall, and F1-score
        precision, recall, f1_score, _ = precision_recall_fscore_support(Y_true,Y_pred, average='macro')

        print("precision=",precision,"recall=",recall,"f1-score=",f1_score,"auc-score=",AUCSCORE)

        Sensitivity =TP/(TP+FN)
        Specificity =TN/(TN+FP)
        fpr = 1-Specificity
        print("sensitivity=",Sensitivity,"specificity=",Specificity,"fpr=",fpr)
        
        import pandas as pd

        data = {'Column1':[precision,recall,f1_score,AUCSCORE,Sensitivity,Specificity,fpr]}
        df_matric = pd.DataFrame(data=data)
        df_matric.index  = ['Precision', 'recall', 'f1-score','Accuracy Score','sensitivity','specificity','fpr']
        print(df_matric)

        
        # Get the number of classes
        num_classes = len(np.unique(Y_true))

        # Create a figure and axis
        # Create a figure and axis
        fig, ax = plt.subplots(figsize=(4,4))
        cmlabels=['Apple','Corn','Grape','Potato','Rice','Tea','Tomato','Wheat']
        # Plot the confusion matrix using a heatmap
        sns.heatmap(confusion, annot=True, cmap="Blues", fmt="d", cbar=False, ax=ax,xticklabels=cmlabels,yticklabels=cmlabels)

        # Set axis labels and title
        ax.set_xlabel("Predicted labels")
        ax.set_ylabel("Actual labels")
        title="CM For VGG19 Model 02 "
        ax.set_title("")
        plt.savefig(title+'confusion_matrix.png')
        # Show the plot
        plt.show()



    def evaluate(self, X, Y_true, metrics):
        Y_pred =self.predict(X)
        #Y_true = Y_true
        
        ret = []
        
        Y_pred=torch.tensor(Y_pred,dtype=torch.float32)


        for m in metrics:
            if m == 'loss':
                loss=[]
                loss = self.binary_cross_entropy(Y_true, Y_pred)
                ret.append(loss)                
            elif m == 'accuracy':
                Y_pred_argmax = torch.argmax(Y_pred, axis=-1)
                Y_true_argmax = torch.argmax(Y_true, axis=-1)
                acc = torch.sum(Y_pred_argmax == Y_true_argmax) / len(Y_true)
                #plotting of confusion matrix

                # Get predicted probabilities for positive class
                Y_true_argmax=Y_true_argmax.cpu()
                Y_pred_argmax=Y_pred_argmax.cpu()
                #self.plot_confusion(Y_true_argmax, Y_pred_argmax)
                ret.append(acc)
            else:
                raise ValueError('an unknown evaluation indicator \'%s\'.' % m)
        if len(ret) == 1:
            print(ret)
            ret = ret
        elif len(ret) == 0:
            ret = None
        return ret

    def fit(self, X, Y_true,Lmax,error,activation):
        device=torch.device("cuda")
        with torch.no_grad():
            self.beta=torch.FloatTensor(np.random.uniform(-1.,1.,size=(1, self.no_output_nodes)))
            self.alpha = torch.FloatTensor(self.no_input_nodes, 1).uniform_(-1,1)
            #print(self.beta.shape, self.alpha.shape,X.shape)
            
            self.alpha=self.alpha.to(device)
            self.beta=self.beta.to(device)
            if(activation=='s'):
                H = torch.sigmoid(torch.matmul(X, self.alpha))
            elif(activation=='t'):
                H = torch.tanh(torch.matmul(X, self.alpha))  
            elif(activation=='r'):
                H = torch.relu(torch.matmul(X, self.alpha))
            
            # Compute a pseudoinverse of H
            H_pinv = torch.pinverse(H)

            # Update beta
            self.beta = torch.matmul(H_pinv, Y_true.float())
            
            for i in range(2, Lmax):
                beta_random = torch.FloatTensor(np.random.uniform(-1.,1.,size=(1, self.no_output_nodes)))
                alpha_random = torch.FloatTensor(self.no_input_nodes, 1).uniform_(-1,1)
                
                beta_random=beta_random.to(device)
                alpha_random=alpha_random.to(device)
                
                self.alpha=torch.cat((self.alpha,alpha_random),dim=1)                          
                self.beta =torch.cat((self.beta,beta_random),dim=0)

                if(activation=='s'):
                    H = torch.sigmoid(torch.matmul(X, self.alpha))
                elif(activation=='t'):
                    H = torch.tanh(torch.matmul(X, self.alpha))  
                elif(activation=='r'):
                    H = torch.relu(torch.matmul(X, self.alpha))
                
                H_pinv = torch.pinverse(H)
                self.beta = torch.matmul(H_pinv, Y_true) 

from  keras.utils import np_utils
def train_features_conversion(x_training_features,y_train_labels):

    # Convert the extracted features to numpy arrays

    device=torch.device("cuda")
    x_training_features_np = x_training_features.detach().cpu().numpy()  
    flattened_features_train = x_training_features_np.reshape(x_training_features_np.shape[0], -1)
    train_features_tensor = torch.tensor(flattened_features_train, dtype=torch.float32).to(device)
    
    y_train_array= y_train_labels.cpu().detach().numpy()
    y_train_array_encoded = np_utils.to_categorical(y_train_array,no_classes)
    y_train_labels_encoded=torch.tensor(y_train_array_encoded,dtype=torch.float32)
    y_train_labels_encoded=y_train_labels_encoded.to(device)    
    return train_features_tensor,y_train_labels_encoded

def test_features_conversion(x_testing_features,y_testing_labels):
    # Convert the extracted features to numpy arrays
    device=torch.device("cuda")
    x_testing_features_np = x_testing_features.detach().cpu().numpy()
    
    flattened_features_test = x_testing_features_np.reshape(x_testing_features_np.shape[0], -1)
    test_features_tensor = torch.tensor(flattened_features_test,dtype=torch.float32).to(device)
    
    y_test_array= y_testing_labels.cpu().detach().numpy()
    y_test_array_encoded = np_utils.to_categorical(y_test_array,no_classes)
    y_test_labels_encoded=torch.tensor(y_test_array_encoded,dtype=torch.float32)
    y_test_labels_encoded=y_test_labels_encoded.to(device)
    return test_features_tensor, y_test_labels_encoded



In [ ]:
device=torch.device("cuda") 
train_path="D:/Smart_Farming_Capstone_Project_G201/Model02/Dataset/Type_of_plant/Train/"
test_path="D:/Smart_Farming_Capstone_Project_G201/Model02/Dataset/Type_of_plant/Test/"
traindata_loader,testdata_loader=loading_data(train_path,test_path)
model=load_model()


In [ ]:
x_training_features,y_train_labels=training_feature_extraction(device,traindata_loader,model)
train_features_tensor,y_train_labels_encoded=train_features_conversion(x_training_features,y_train_labels)
torch.save(train_features_tensor,"D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Train/train_featuresVGG19.pth")
torch.save(y_train_labels_encoded,"D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Train/train_labelsVGG19.pth")

In [ ]:
x_testing_features,y_testing_labels=testing_feature_extratcion(device,testdata_loader,model)
test_features_tensor, y_test_labels_encoded=test_features_conversion(x_testing_features,y_testing_labels)
torch.save(test_features_tensor,"D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Test/testing_featuresVGG19.pth")
torch.save(y_test_labels_encoded,"D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Test/testing_labelsVGG19.pth")

In [ ]:
def i_elmTraining(features_train,y_train,activation):
  # ===============================
  # Instantiate ELM object
  # ===============================
  input=features_train.size(1)
  ielm_model = I_ELM(
      no_input_nodes=input,
      max_no_hidden_nodes=Lmax,
      no_output_nodes=no_classes,
      loss_function=loss_function,
      activation_function=activation_function,    
  )

  i = time.time()  
  ielm_model.fit(features_train,y_train,Lmax,error,activation)
  final = time.time()

  training_loss, training_acc = ielm_model.evaluate(features_train, y_train, metrics=['loss','accuracy'])
  
  print('Training Loss in mean square error: %f' % training_loss) # loss value
  print('Training Accuracy: %f' % training_acc)# accuracy
  print('Total Time require for Training %f Seconds'% (final-i))

  return ielm_model
def i_elmTesting(ielm_model,features_test,y_test):
  
  i = time.time()
  test_loss, test_acc = ielm_model.evaluate(features_test, y_test, metrics=['loss', 'accuracy'])
  final = time.time()
  
  print('Testing Loss in mean square error: %f ' % test_loss,"\n")
  print('Testing Accuracy: %f' % test_acc)
  time_taken=(final-i)/features_test.shape[0]
  print('The total testing time is ',(final-i))
  print('Total Time require for Testing one image is ' ,time_taken ,'seconds')
  


In [ ]:
x_training_features=torch.load("D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Train/train_featuresVGG19.pth")
y_train_labels=torch.load("D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Train/train_labelsVGG19.pth")


In [ ]:

x_testing_features=torch.load("D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Test/testing_featuresVGG19.pth")
y_testing_labels=torch.load("D:/Smart_Farming_Capstone_Project_G201/Model02/Feautres_Extraction/VGG19/Test/testing_labelsVGG19.pth")



In [13]:
Lmax=251
input=x_training_features.size(1)
ielm_model = I_ELM(
no_input_nodes=input,
max_no_hidden_nodes=Lmax,
no_output_nodes=no_classes,
loss_function=loss_function,
activation_function=activation_function,    
)
activation='s'
for i in range(Lmax):
    ielm_model.fit(x_training_features,y_train_labels,i,error,activation)
    testing_loss, testing_acc = ielm_model.evaluate(x_testing_features, y_testing_labels, metrics=['loss','accuracy'])
    #print("neurons=",i,'testing Loss in mean square error: %f ' % testing_loss,i) # loss value
    print("no of neuron=",i,'Testing Accuracy: %f' % testing_acc)# accuracy

#print('testing Loss in mean square error: %f' % testing_loss) # loss value
#print('Testing Accuracy: %f' % testing_acc)# accuracy

C:\Users\Dr. Rajendra Ku Roul\AppData\Local\Temp\ipykernel_13544\2997670534.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self(X), dtype=torch.float32)
C:\Users\Dr. Rajendra Ku Roul\AppData\Local\Temp\ipykernel_13544\2997670534.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y_pred=torch.tensor(Y_pred,dtype=torch.float32)


no of neuron= 0 Testing Accuracy: 0.125444
no of neuron= 1 Testing Accuracy: 0.123000
no of neuron= 2 Testing Accuracy: 0.126667
no of neuron= 3 Testing Accuracy: 0.178111
no of neuron= 4 Testing Accuracy: 0.197111
no of neuron= 5 Testing Accuracy: 0.262000
no of neuron= 6 Testing Accuracy: 0.241444
no of neuron= 7 Testing Accuracy: 0.243889
no of neuron= 8 Testing Accuracy: 0.297444
no of neuron= 9 Testing Accuracy: 0.312333
no of neuron= 10 Testing Accuracy: 0.300333
no of neuron= 11 Testing Accuracy: 0.339889
no of neuron= 12 Testing Accuracy: 0.363111
no of neuron= 13 Testing Accuracy: 0.361667
no of neuron= 14 Testing Accuracy: 0.364333
no of neuron= 15 Testing Accuracy: 0.351333
no of neuron= 16 Testing Accuracy: 0.380778
no of neuron= 17 Testing Accuracy: 0.416444
no of neuron= 18 Testing Accuracy: 0.409111
no of neuron= 19 Testing Accuracy: 0.383667
no of neuron= 20 Testing Accuracy: 0.398667
no of neuron= 21 Testing Accuracy: 0.463111
no of neuron= 22 Testing Accuracy: 0.48822

In [ ]:
Lmax=250
activation='s'
i_elm_model=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model,x_testing_features,y_testing_labels)

In [ ]:
Lmax=300
activation='s'
i_elm_model1=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model1,x_testing_features,y_testing_labels)

In [ ]:
Lmax=250
activation='s'
i_elm_model=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model,x_testing_features,y_testing_labels)

In [ ]:
Lmax=100
activation='s'
i_elm_model=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model,x_testing_features,y_testing_labels)

In [ ]:
Lmax=400
activation='t'
i_elm_model2=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model2,x_testing_features,y_testing_labels)

In [ ]:
Lmax=300
activation='t'
i_elm_model2=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model2,x_testing_features,y_testing_labels)

In [ ]:
Lmax=250
activation='t'
i_elm_model2=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model2,x_testing_features,y_testing_labels)

In [ ]:
Lmax = 100
activation='t'
i_elm_model2=i_elmTraining(x_training_features,y_train_labels,activation)

In [ ]:
i_elmTesting(i_elm_model2,x_testing_features,y_testing_labels)

In [ ]:
y_train_labels_reshaped=np.argmax(y_train_labels.detach().cpu().numpy(),axis=1)


In [ ]:
y_test_labels_reshaped=np.argmax(y_testing_labels.detach().cpu().numpy(),axis=1)

GNB

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Initialize and train the naive bayes classifier
naiveClassifier=GaussianNB(var_smoothing=1e-8)

start_time=time.time()
naiveClassifier.fit(x_training_features.cpu(),y_train_labels_reshaped)
end_time=time.time()

print("training_time =", end_time-start_time)

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
testing_start_time=time.time()
y_pred=naiveClassifier.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time=", testing_stop_time-testing_start_time)
# Evaluate the classifier
accuracy = accuracy_score(y_test_labels_reshaped,y_pred)
print("Accuracy:", accuracy)

precision_1 = precision_score(y_test_labels_reshaped, y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped, y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped, y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)
cm = confusion_matrix(y_test_labels_reshaped, y_pred)

# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity)
print("sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)


SVM

In [ ]:
from sklearn.svm import SVC
import time
from sklearn.model_selection import GridSearchCV
svm=SVC(C=1,kernel='linear',gamma='auto')

start_time= time.time()
#svm=svm.cuda().to(device)
svm.fit(x_training_features.cpu(),y_train_labels_reshaped)
end_time=time.time()
print("training time=",end_time-start_time)


In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
testing_start_time=time.time()
y_pred=svm.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time=", testing_stop_time-testing_start_time)
# Evaluate the classifier
accuracy = accuracy_score(y_test_labels_reshaped,y_pred)
print("Accuracy:", accuracy)

precision_1 = precision_score(y_test_labels_reshaped, y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped, y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped, y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)
cm = confusion_matrix(y_test_labels_reshaped, y_pred)

# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity,"sensivity=",sensitivity)

TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)

Random Forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
rf_classifier = RandomForestClassifier(n_estimators=100)
start_time= time.time()
rf_classifier.fit(x_training_features.cpu(),y_train_labels_reshaped)
end_time=time.time()
print("training time=",end_time-start_time)


In [ ]:
# Make predictions on the testing set
from sklearn.metrics import accuracy_score
testing_start_time=time.time()
y_pred = rf_classifier.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time=", testing_stop_time-testing_start_time)

# Evaluate the classifier
accuracy = accuracy_score(y_test_labels_reshaped, y_pred)
print("Accuracy:", accuracy)
from sklearn.metrics import precision_score,recall_score,f1_score
precision_1 = precision_score(y_test_labels_reshaped, y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped, y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped, y_pred,average='macro')

cm = confusion_matrix(y_test_labels_reshaped, y_pred)
# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]
#print(tp,fp)

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)

print("Specificity:", specificity)
print("sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)

Decision tree classifier    

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Create a Decision Tree classifier
dt_classifier = DecisionTreeClassifier(criterion='gini')

start_time= time.time()
dt_classifier.fit(x_training_features.cpu(),y_train_labels_reshaped)
end_time=time.time()
print("training time=",end_time-start_time)


In [ ]:

# Make predictions on the testing set
from sklearn.metrics import accuracy_score
testing_start_time=time.time()
y_pred = dt_classifier.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time=", testing_stop_time-testing_start_time)

# Evaluate the classifier
accuracy = accuracy_score(y_test_labels_reshaped,y_pred)
print("Accuracy:", accuracy)
from sklearn.metrics import precision_score,recall_score,f1_score
precision_1 = precision_score(y_test_labels_reshaped,y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped,y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped,y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)

cm = confusion_matrix(y_test_labels_reshaped,y_pred)
#print(cm)
# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]
#print(tp,fp)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity)
print("sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)


ET Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

# Create the Extra Trees classifier
clf = ExtraTreesClassifier(criterion="gini")

start_time=time.time()
clf.fit(x_training_features.cpu(), y_train_labels_reshaped)
#model.fit(, )
stop_time=time.time()
print("trainingtime=",stop_time-start_time)


In [ ]:

testing_start_time=time.time()
y_pred = clf.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time =", testing_stop_time-testing_start_time)

# Calculate accuracy score
accuracy = accuracy_score(y_test_labels_reshaped, y_pred)

print("accuracy:",accuracy)
from sklearn.metrics import precision_score,recall_score,f1_score
precision_1 = precision_score(y_test_labels_reshaped,y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped,y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped,y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)

cm = confusion_matrix(y_test_labels_reshaped,y_pred)
#print(cm)
# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]
#print(tp,fp)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity)
print("sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)




In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score

# Create the Extra Trees classifier
clf1 = ExtraTreesClassifier(criterion="entropy")

start_time=time.time()
clf1.fit(x_training_features.cpu(), y_train_labels_reshaped)
#model.fit(, )
stop_time=time.time()
print("trainingtime=",stop_time-start_time)


In [ ]:

testing_start_time=time.time()
y_pred = clf1.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time =", testing_stop_time-testing_start_time)

# Calculate accuracy score
accuracy = accuracy_score(y_test_labels_reshaped, y_pred)

print("accuracy:",accuracy)
from sklearn.metrics import precision_score,recall_score,f1_score
fscore=f1_score(y_test_labels_reshaped, y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)


# Assuming y_true and y_pred are your true and predicted labels, respectively
# Assuming y_true and y_pred are your true and predicted labels, respectively
cm = confusion_matrix(y_test_labels_reshaped, y_pred)
#print(cm)
# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]
#print(tp,fp)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity)
print("sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,TPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)



XGBoost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
model = xgb.XGBClassifier(
    n_estimators=100,  # Number of boosting rounds
    max_depth=3,  # Maximum depth of each tree
    learning_rate=0.001,  # Learning rate (step size shrinkage)
    objective='multi:softprob',  # Objective function for binary classification
    random_state=42,n_jobs=4,booster ="gbtree", gamma = 0
)
start_time=time.time()
model.fit(x_training_features.cpu(), y_train_labels_reshaped)
stop_time=time.time()
print("trainingtime=",stop_time-start_time)


In [ ]:

testing_start_time=time.time()
y_pred = model.predict(x_testing_features.cpu())
testing_stop_time=time.time()
print("testing time =", testing_stop_time-testing_start_time)
report = classification_report(y_test_labels_reshaped, y_pred)
print(report)
# Calculate accuracy score
accuracy = accuracy_score(y_test_labels_reshaped, y_pred)

print("accuracy:",accuracy)
from sklearn.metrics import precision_score,recall_score,f1_score
precision_1 = precision_score(y_test_labels_reshaped, y_pred,average='macro')
recall=recall_score(y_test_labels_reshaped, y_pred,average='macro')
fscore=f1_score(y_test_labels_reshaped, y_pred,average='macro')
print("precision:",precision_1)
print("recall",recall)
print("f1-Score",fscore)


# Assuming y_true and y_pred are your true and predicted labels, respectively
# Assuming y_true and y_pred are your true and predicted labels, respectively
cm = confusion_matrix(y_test_labels_reshaped, y_pred)
print(cm)
# Extract true positives (TP), false positives (FP), false negatives (FN), and true negatives (TN) from the confusion matrix
tp = cm[1, 1]
fp = cm[0, 1]
fn = cm[1, 0]
tn = cm[0, 0]
#print(tp,fp)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
print("Specificity:", specificity,"sensivity=",sensitivity)
TPR= tp/(tp+fn)
FPR=fp/(fp+tn)
X=[0,FPR,1]
Y=[0,FPR,1]
AUCSCORE =np.trapz(Y,X)
print("AUCSCORE=", AUCSCORE)

